## Calculations in F_2 P

This script aims to create an environment correspinding to F_2 P, where one can carry out all the calculations that one could want: inverting elements of P as well as adding and multiplying elements of F_2 P. In order to do this, some functions from 'Generating balls of P and finding equals' are copyed here (as importing got messy) and the balls up to B(10) imported.

Any element in F_2 P will be presented by a list of elements of P

Secondly this script also aims to translate elements between the string as named in 'Generating Balls of P and finding equals' and the integers that is the SAT-solvers input and output 

## The setup

### Copyed functions and lists

In [7]:
inverses=['qw','wq','po','op']
all_relation_identities=['wppqpp', 'ppqppw', 'pqppwp', 'qppwpp', 'ppwppq', 'pwppqp', 'oowooq', 'owooqo', 'wooqoo',
                         'ooqoow', 'oqoowo', 'qoowoo', 'oqqpqq', 'qqpqqo', 'qpqqoq', 'pqqoqq', 'qqoqqp', 'qoqqpq', 
                         'wwowwp', 'wowwpw', 'owwpww', 'wwpwwo', 'wpwwow', 'pwwoww']

In [8]:
def remove_identity(w):
    '''The function removes any instance of 28 identity words
    - the 4 words of length 2 and the 24 of length 6 given by the realtion on P.
    The function needs the globally defined lists all_relation_identities and inverses'''
    word=w

    #mapping inverses to the identity
    for i in inverses:
        word=word.replace(i,'')

    if len(word)>=6: #Possibly saving some time
        #Doing the same for all the identities from the relations
    
        for i in all_relation_identities:
            word=word.replace(i,'')
    return word

In [9]:
def repeat_remove_identity(w):
    '''This function repeatedly runs remove_identity, untill it does not have any of the 28 identity words in it.
    It requires remove_identity function as well as the two lists of identities
    '''
    word=w

    #The word is reduced by the remove_identity function
    word=remove_identity(word)

    #If the word is unchanged by the function above it is simply returned
    if word==w:
        return word        

    #otherwise it calls this function again
    return repeat_remove_identity(word)

In [10]:
def inverse(w):
    '''This function returns the inverse element of w'''
    #Inversed order
    word=w[::-1]

    #Swapping all a and o's as well as p and b's (via dummies called e)
    word=word.replace('p','e')
    word=word.replace('o','p')
    word=word.replace('e','o')
    word=word.replace('w','e')
    word=word.replace('q','w')
    word=word.replace('e','q')

    return word

In [11]:
sisters=['wpp', 'ppq', 'pqp', 'qpp', 'ppw', 'pwp', 'oow', 'owo', 'woo', 'ooq', 'oqo', 'qoo', 'oqq',
         'qqp', 'qpq', 'pqq', 'qqo', 'qoq', 'wwo', 'wow', 'oww', 'wwp', 'wpw', 'pww']
sistertwin=['oow', 'qoo', 'oqo', 'ooq', 'woo', 'owo', 'wpp', 'pwp', 'ppw', 'qpp', 'pqp', 'ppq', 'wwo',
            'pww', 'wpw', 'wwp', 'oww', 'wow', 'oqq', 'qoq', 'qqo', 'pqq', 'qpq', 'qqp']
sisters_special=['wpp', 'ppq', 'pqp', 'qpp', 'ppw', 'pwp', 'oqq', 'qqp', 'qpq', 'pqq', 'qqo', 'qoq',
                 'oow', 'owo', 'woo', 'ooq', 'oqo', 'qoo', 'wwo', 'wow', 'oww', 'wwp', 'wpw', 'pww']

In [12]:
def reduce(w):
    ''' This function reduces w by running the repeat_remove_identity and then
    swapping sisters following the order of the sisters_special list, reducing for identity-occurences along the way
    '''
    word=w

    word=repeat_remove_identity(word)
    
    for i in sisters_special:
        word=word.replace(i,sistertwin[sisters.index(i)])
        word=repeat_remove_identity(word)

    return word

In [13]:
import random

In [14]:
def reduce_random(w,n):
    ''' This function reduces w by running the repeat_remove_identity and then
    randomly (via random.randint) swapping sisters n times, reducing for identity-occurences along the way'''

    word=w
    word=repeat_remove_identity(word)
    
    for i in range(n):
        r=random.randint(0,23)
        word=word.replace(sisters[r],sistertwin[r])
        word=repeat_remove_identity(word)

    return word

In [15]:
def find_equal(w,lst,n=100,r=1):
    ''' This function reduces the word w recursively (via both reduce and reduce_random) until it is found in the lst. 
        If it has not found a solution after 1000 runs it returns 'problem' '''
    word=w
    word=reduce(word)
    word=reduce_random(word,n)
    
    if word in lst:
        return word
    
    if r>1001:
        return 'problem'
    
    return find_equal(word,lst,n,r+1)

### Imported balls B(10)

In [17]:
import ast #helps to change string into list below

In [18]:
f = open("B.txt")
B=ast.literal_eval(f.read())

### Defining sum and product function

Sum is defined quite simply as joining lists. And this is already a feature of python as following example illustrates

In [21]:
['p','pq','']+['p']

['p', 'pq', '', 'p']

We therefore only need the sum function to check that no string occurs twice in the adjoined list.

In [23]:
def sum(*arg):
    '''This function joins lists and removes all copys of elements, so elements only occur and only occur once if it occurs an odd number
        of times on the joined list. Note, to work as the sum of F_2 P, the inputted lists must be reduced (no even multi-occurences of terms)'''
    sum_candidate=[item for sublist in [*arg] for item in sublist] #lists joined
    sum_done=[w for w in set(sum_candidate) if sum_candidate.count(w)%2==1] #only 1 copy of each string with an odd occurence is included in the product
    return sum_done

Now, we define a product function

In [25]:
import itertools
def product(w1,w2,n=10):
    '''This function takes two lists of elements in P - corresponding to elements in F_2 P and outputs the list corresponding to their product'''
    cart_product=[''.join(word) for word in itertools.product(w1, w2)] # Cartesian product generated
    reduced_product=[find_equal(word,B[n]) for word in cart_product] # Every string reduced to unique form in B[n] - obs default n is 10
    true_product=[w for w in set(reduced_product) if reduced_product.count(w)%2==1]# Reducing mod 2
    if 'problem' in reduced_product: #Important warning if not all elements have been successfully reduced
        print('NOTE: problem in finding equals')
    return true_product

### Translation function

In [27]:
def translate(lst,n=5,alpha=True):
    '''Function that translates a list of strings representing elements of P to the integers that SAT has as input/output and vice versa.
    n denotes which ball SAT is operating within. alpha=True signifies that we have focus on integers btw 1 and |B(n)| 
    and alpha=False signifies a focus on integers btw |B(n)|+1 and 2|B(n)| corresponding to the elements alpha and beta respectively in the thesis section on translation.
    This function uses the lists B[n]'''

    global B
    
    if type(lst[0])==int: #Translation from integers to strings. Note the code only 'sees' positive input. Reads alpha or beta depending on alpha=True og alpha=False
        if alpha==True:
            new_lst=[i-1 for i in lst if i>0 and i<len(B[n])+1]
        if alpha==False:
            new_lst=[i-len(B[n])-1 for i in lst if i>len(B[n]) and i<2*len(B[n])+1]
            
        translation=[B[n][i] for i in new_lst]
        
    if type(lst[0])==str: #Translation from strings to integers. Note it depends on wheter this element should act as alpha or beta (i.e. first or second element in product)
        if alpha==True:
            translation=[B[n].index(w)+1 for w in lst]
        if alpha==False:
            translation=[B[n].index(w)+1+len(B[n]) for w in lst]

    return translation

## Time to play

Now elements can be added and multiplied

In [448]:
sum(['p','q'],[inverse('pq'),'p'])

['q', 'wo']

In [479]:
product(['p','q'],[inverse('pq'),'p'])

['pp', 'o', 'pwo', 'qp']

### Testing counterexample from model with translater

Now, lets see that the counterexample does indeed multiply to 1. First, we import the counterexample from the model.txt file.

In [43]:
g = open("model.txt")
model=ast.literal_eval(g.read())

In [45]:
alpha=translate(model,4,True)
beta=translate(model,4,False)
product(alpha,beta)

['']

In [485]:
product(beta,alpha) #Other order

['']

In [487]:
print(alpha)
print(beta)

['qpq', 'pw', 'qqq', 'qoq', 'wo', 'qpp', 'p', 'qqqp', 'qpw', 'qpwp', 'ppq', 'pp', 'pwp', 'qqo', 'qpqp', 'qow', 'o', 'ow', 'qqp', 'ooqq', 'w']
['pqpw', 'pqp', 'pqpq', 'qpq', 'q', 'ppw', 'qoq', 'wppw', 'p', 'qp', 'qpw', 'pp', 'qqo', 'qo', 'oq', 'qow', 'qqpw', 'o', 'www', 'qqp', 'wpp']


Now we manipulate the lists to print in thesis rapport

In [41]:
alpha_string=''
for w in sorted(alpha,key=len):
    w=w.replace('o','p^{-1}')
    w=w.replace('w','q^{-1}')
    alpha_string=alpha_string+w+' + '
print(alpha_string)

p + p^{-1} + q^{-1} + pq^{-1} + q^{-1}p^{-1} + pp + p^{-1}q^{-1} + qpq + qqq + qp^{-1}q + qpp + qpq^{-1} + ppq + pq^{-1}p + qqp^{-1} + qp^{-1}q^{-1} + qqp + qqqp + qpq^{-1}p + qpqp + p^{-1}p^{-1}qq + 


In [42]:
beta_string=''
for w in sorted(beta,key=len):
    w=w.replace('o','p^{-1}')
    w=w.replace('w','q^{-1}')
    beta_string=beta_string+w+' + '
print(beta_string)

q + p + p^{-1} + qp + pp + qp^{-1} + p^{-1}q + pqp + qpq + ppq^{-1} + qp^{-1}q + qpq^{-1} + qqp^{-1} + qp^{-1}q^{-1} + q^{-1}q^{-1}q^{-1} + qqp + q^{-1}pp + pqpq^{-1} + pqpq + q^{-1}ppq^{-1} + qqpq^{-1} + 


### Checking Gardams counterexample

Gardam defines his counterexample by some intermediate steps

In [47]:
x_Gardam='pp'
y_Gardam='qq'
z_Gardam='pqpq'

In [49]:
p_Gardam=product(product(['',x_Gardam],['',y_Gardam]),['',inverse(z_Gardam)]) #(1+x)(1+y)(1+z^-1)
q_Gardam=sum(product([inverse(x_Gardam)],[inverse(y_Gardam)]),[x_Gardam],product([inverse(y_Gardam)],[z_Gardam]),[z_Gardam])
r_Gardam=sum(['',x_Gardam],product([inverse(y_Gardam)],[z_Gardam]),product(product([x_Gardam],[y_Gardam]),[z_Gardam]))
s_Gardam=sum([''],product([x_Gardam,inverse(x_Gardam),y_Gardam,inverse(y_Gardam)],[inverse(z_Gardam)]))

In [51]:
unit_Gardam=sum(p_Gardam,product(q_Gardam,['p']),product(r_Gardam,['q']),product(s_Gardam,['pq']))

Now, here is Gardams unit:

In [53]:
print(unit_Gardam)

['', 'qowo', 'ppwo', 'ppp', 'qowp', 'wp', 'qo', 'qqpwo', 'qq', 'woqq', 'qqpp', 'oqo', 'wowo', 'qoqp', 'pp', 'pq', 'pwpwp', 'oqq', 'pqowo', 'q', 'ppq']


In [55]:
pp_Gardam=product(product([inverse(x_Gardam)+'o'],p_Gardam),['p']) #Note the '+' in here is a multiplication btw elements of P
qq_Gardam=product([inverse(x_Gardam)],q_Gardam) #He has an additive inverse here (-), but it doesn't make sense if the field has charachter 2
rr_Gardam=product([inverse(y_Gardam)],r_Gardam) #He has an additive inverse here (-), but it doesn't make sense if the field has charachter 2
ss_Gardam=product([inverse(z_Gardam)+'o'],product(s_Gardam,['p'])) #Note the '+' in here is a multiplication btw elements of P

In [57]:
inverse_Gardam=sum(pp_Gardam,product(qq_Gardam,['p']),product(rr_Gardam,['q']),product(ss_Gardam,['pq']))

And the inverse

In [59]:
print(inverse_Gardam)

['', 'oqpw', 'oqow', 'oqqo', 'wwpq', 'woo', 'oqpq', 'oqqqo', 'oqpwp', 'pwo', 'w', 'oo', 'ww', 'oqoqo', 'p', 'pw', 'pqoo', 'pqpq', 'wppwo', 'wo', 'oq']


And the product (both orders)

In [61]:
product(unit_Gardam,inverse_Gardam)

['']

In [64]:
product(inverse_Gardam,unit_Gardam)

['']

### Translating Gardams unit into SAT

Note, due to the way the balls are constructed in the code of 'Generating Balls..', each ball is not a subset of the next. As seen here:

In [55]:
for i in range(len(B)-1):
    print(set(B[i]) <= set(B[i+1]))

True
True
True
True
False
False
False
False
False
False


In other words, the representative for each element is not the same in the different balls. The products calculated in the section above here refer to the ball B[10], but when translating into SAT, we need representatives of B[5]. So lets firstly check, if there are any elements in the support of Gardams unit, that is represented by a word, which is not in the list B[5].

In [57]:
[w for w in unit_Gardam if w not in B[5]]

['qqpp', 'wowo', 'qowo', 'pwpwp', 'pqowo', 'oqq', 'woqq', 'ppwo']

Now the fitting representatives are found and substituted into Gardam's unit

In [59]:
[find_equal(w,B[5]) for w in unit_Gardam if w not in B[5]]

['pwwp', 'qpqp', 'qpwp', 'pqpqp', 'oqowp', 'wwo', 'wwwo', 'wooo']

In [60]:
unit_Gardam=[find_equal(w,B[5]) if w not in B[5] else w for w in unit_Gardam]

Now, the translate function can be used

In [62]:
print(translate(unit_Gardam))

[1, 84, 89, 93, 64, 5, 143, 67, 3, 83, 20, 118, 31, 66, 68, 49, 80, 56, 26, 58, 61]


And this can be put into SAT, so we export a list of clauses of length 1

In [64]:
clauses_unit_Gardam=[[w] for w in translate(unit_Gardam)]

In [65]:
with open('clauses_unit_Gardam.txt','w') as h:
        h.write(str(clauses_unit_Gardam))

Same process is done for the inverse

In [67]:
[w for w in inverse_Gardam if w not in B[5]]

['oqpq', 'pqpq', 'oqpwp', 'oqoqo']

In [68]:
[find_equal(w,B[5]) for w in inverse_Gardam if w not in B[5]]

['owpw', 'pwpw', 'pqpwo', 'owowo']

In [69]:
inverse_Gardam=[find_equal(w,B[5]) if w not in B[5] else w for w in inverse_Gardam]

In [70]:
print(translate(inverse_Gardam,alpha=False))

[148, 259, 191, 294, 239, 279, 185, 219, 225, 201, 255, 286, 190, 288, 238, 283, 172, 237, 174, 222, 186]


In [71]:
clauses_inverse_Gardam=[[w] for w in translate(inverse_Gardam,alpha=False)]

In [72]:
with open('clauses_inverse_Gardam.txt','w') as h2:
        h2.write(str(clauses_inverse_Gardam))

### Relation btw. counterexamples

These are some simple investigations on the different counterexamples - especially, we are looking for a link between the counterexamples.

In [183]:
print(alpha)
print(unit_Gardam)

['qpq', 'pw', 'qqq', 'qoq', 'wo', 'qpp', 'p', 'qqqp', 'qpw', 'qpwp', 'ppq', 'pp', 'pwp', 'qqo', 'qpqp', 'qow', 'o', 'ow', 'qqp', 'ooqq', 'w']
['', 'qoqp', 'pwwp', 'qpqp', 'qpwp', 'qowp', 'qq', 'pp', 'pqpqp', 'ppp', 'oqowp', 'wwo', 'q', 'ppq', 'oqo', 'qqpwo', 'qo', 'wwwo', 'wp', 'wooo', 'pq']


In [185]:
print(len(alpha))
print(len(unit_Gardam))

21
21


Relatively naive search: See if the counterexample occurs by multiplying alpha/beta by the inverse of any element in the support. Note both right/left multiplication should be tested.

In [168]:
for w in alpha:
    test1=sorted(product([inverse(w)],alpha))
    test2=sorted(product(alpha,[inverse(w)]))
    value1=sorted(unit_Gardam)==test1
    value2=sorted(unit_Gardam)==test2
    if value1 or value2:
        print('------YES-------')
    print(w,value1,value2)

for w in beta:
    test1=sorted(product([inverse(w)],beta))
    test2=sorted(product(beta,[inverse(w)]))
    value1=sorted(unit_Gardam)==test1
    value2=sorted(unit_Gardam)==test2
    if value1 or value2:
        print('------YES-------')
    print(w,value1,value2)

qpq False False
pw False False
qqq False False
qoq False False
wo False False
qpp False False
p False False
qqqp False False
qpw False False
qpwp False False
ppq False False
pp False False
pwp False False
qqo False False
qpqp False False
qow False False
o False False
ow False False
qqp False False
ooqq False False
w False False
pqpw False False
pqp False False
pqpq False False
qpq False False
q False False
ppw False False
qoq False False
wppw False False
p False False
qp False False
qpw False False
pp False False
qqo False False
qo False False
oq False False
qow False False
qqpw False False
o False False
www False False
qqp False False
wpp False False


Okay, we see that the element is not immediately in relation to Gardams inverse. But we try the same test for conjugations of Gardams inverse by words of length 1.

In [177]:
conjugations=[]
for w in ['p','o','q','w']:
    conjugations.append(sorted(product(product([w],unit_Gardam),[inverse(w)])))
    conjugations.append(sorted(product(product([inverse(w)],unit_Gardam),[w])))
print(conjugations)

[['', 'oqow', 'pp', 'ppp', 'pppqo', 'ppq', 'ppqo', 'pqo', 'pqoo', 'pqoq', 'pqow', 'pqpq', 'pw', 'pwoooo', 'qoq', 'qow', 'qqo', 'qqpwoo', 'woqqooo', 'ww', 'wwpp'], ['', 'oq', 'oqo', 'oqow', 'oqp', 'owpp', 'pp', 'ppp', 'pqoq', 'pqow', 'pqpq', 'pw', 'qoq', 'qow', 'qp', 'qpp', 'qqo', 'qqow', 'ww', 'wwpp', 'www'], ['', 'oq', 'oqo', 'oqow', 'oqp', 'owpp', 'pp', 'ppp', 'pqoq', 'pqow', 'pqpq', 'pw', 'qoq', 'qow', 'qp', 'qpp', 'qqo', 'qqow', 'ww', 'wwpp', 'www'], ['', 'oqow', 'pp', 'ppp', 'pppqo', 'ppq', 'ppqo', 'pqo', 'pqoo', 'pqoq', 'pqow', 'pqpq', 'pw', 'pwoooo', 'qoq', 'qow', 'qqo', 'qqpwoo', 'woqqooo', 'ww', 'wwpp'], ['', 'oo', 'ooqpw', 'oq', 'oqpq', 'owpp', 'owpq', 'owwo', 'pqpq', 'pw', 'pwpq', 'q', 'qoq', 'qoqoqpw', 'qoqow', 'qp', 'qpp', 'qpwoq', 'qq', 'qqow', 'wowoqpq'], ['', 'oo', 'oq', 'oqpq', 'oqqooq', 'owpq', 'owwo', 'pqpq', 'pwpq', 'q', 'qoqow', 'qpp', 'qpwoq', 'qq', 'wowowpq', 'wpqoo', 'wpqowoq', 'wpqq', 'wwoqqq', 'wwpq', 'wwwoq'], ['', 'oo', 'oq', 'oqpq', 'oqqooq', 'owpq', 'owwo'

Looking at the lengths of words in Gardams unit + inverse and the unit found in the other script + inverse

In [204]:
candidates=[alpha,beta,unit_Gardam,inverse_Gardam]
for elem in candidates:
    print(sorted([len(w) for w in elem]))

[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[0, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5]
[0, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5]


Seeing if Gardam's unit can be translated into an element with support within B(4)

In [226]:
for w1 in B[3]:
    for w2 in B[3]:
        elem=product(product([w1],unit_Gardam),[w2])
        max_length=max([len(w) for w in elem])
        if max_length<5:
            print(w1,w2)

NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
o oqq
o oq
o p
o pq
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in finding equals
NOTE: problem in fin

In [286]:
interesting_objects=[]
w1='o'
for w2 in ['oqq','oq','p','pq']:
     interesting_objects.append(sorted(product(product([w1],unit_Gardam),[w2]),key=len))
print(interesting_objects)

[['', 'oo', 'pw', 'wo', 'oq', 'qq', 'ooo', 'qqq', 'owo', 'qpq', 'wwp', 'wpq', 'woo', 'pwo', 'pwpq', 'owwo', 'wwpq', 'pqpq', 'owpq', 'pqoo', 'oqpq'], ['o', 'q', 'w', 'qp', 'qq', 'wp', 'pq', 'oqp', 'wpp', 'owo', 'ppp', 'oqo', 'wwp', 'qoq', 'owp', 'pww', 'qpp', 'pqoq', 'pqpq', 'wwpp', 'owpp'], ['', 'qp', 'ww', 'pw', 'pp', 'oq', 'oqp', 'ppp', 'oqo', 'qqo', 'qow', 'qoq', 'www', 'qpp', 'pqoq', 'oqow', 'pqpq', 'pqow', 'wwpp', 'qqow', 'owpp'], ['p', 'w', 'q', 'qo', 'ww', 'ow', 'wo', 'oqq', 'ooo', 'owo', 'ppq', 'oqo', 'pqo', 'qpq', 'qqo', 'woo', 'pwo', 'owwo', 'pqpq', 'pqoo', 'oqpq']]


In [284]:
interesting_objects_inverses=[]
w1='p'
for w2 in ['oqq','oq','p','pq']:
     interesting_objects_inverses.append(sorted(product(product([inverse(w2)],inverse_Gardam),[w1]),key=len))
for lst in interesting_objects_inverses:
    print(max([len(w) for w in lst]))

5
4
5
4


In [255]:
for i in range(4):
    print(product(interesting_objects[i],interesting_objects_inverses[i]))

['']
['']
['']
['']


In [261]:
for i in range(4):
    print(interesting_objects_inverses[i])

['', 'o', 'oqo', 'pp', 'ppwo', 'ppwww', 'pq', 'pqo', 'q', 'qo', 'qoq', 'qoqp', 'woowp', 'woq', 'woqp', 'woqq', 'wowo', 'wp', 'wpqp', 'ww', 'wwpp']
['ooo', 'oq', 'oqo', 'oqp', 'oqq', 'oqqo', 'ow', 'owo', 'owp', 'p', 'q', 'qo', 'qpp', 'qpq', 'qpqo', 'qq', 'qqo', 'w', 'wowo', 'wpp', 'wppp']
['', 'oo', 'ooqp', 'ow', 'owo', 'owp', 'owwo', 'owwoo', 'p', 'qo', 'qowo', 'qpq', 'qpqo', 'qpw', 'qpwo', 'qq', 'qqqp', 'qqqpp', 'w', 'wowo', 'wp']
['o', 'oqo', 'owo', 'ppp', 'ppq', 'ppqo', 'pq', 'pqo', 'pw', 'pwo', 'pww', 'q', 'qoq', 'qoqp', 'qqpp', 'w', 'woo', 'wowo', 'wp', 'ww', 'wwp']


In [288]:
vio=[interesting_objects[1],interesting_objects[3]]
vio_inverse=[interesting_objects_inverses[1],interesting_objects_inverses[3]]

In [290]:
for i in range(2):
    print(product(vio[i],vio_inverse[i]))
    print(max([len(w) for w in vio[i]]))
    print(max([len(w) for w in vio_inverse[i]]))
    print(vio[i])
    print(vio_inverse[i])

['']
4
4
['o', 'q', 'w', 'qp', 'qq', 'wp', 'pq', 'oqp', 'wpp', 'owo', 'ppp', 'oqo', 'wwp', 'qoq', 'owp', 'pww', 'qpp', 'pqoq', 'pqpq', 'wwpp', 'owpp']
['p', 'w', 'q', 'qo', 'ow', 'oq', 'qq', 'oqp', 'oqq', 'ooo', 'wpp', 'owo', 'oqo', 'qpq', 'qqo', 'owp', 'qpp', 'oqqo', 'wppp', 'wowo', 'qpqo']
['']
4
4
['p', 'w', 'q', 'qo', 'ww', 'ow', 'wo', 'oqq', 'ooo', 'owo', 'ppq', 'oqo', 'pqo', 'qpq', 'qqo', 'woo', 'pwo', 'owwo', 'pqpq', 'pqoo', 'oqpq']
['o', 'q', 'w', 'ww', 'pw', 'wp', 'pq', 'owo', 'ppq', 'ppp', 'oqo', 'pqo', 'wwp', 'woo', 'qoq', 'pww', 'pwo', 'qoqp', 'qqpp', 'wowo', 'ppqo']


Lengths of words in these two new solutions + the counterexample found by SAT

In [339]:
candidates=[alpha,beta,vio[0],vio_inverse[0],vio[1],vio_inverse[1]]
for elem in candidates:
    print(sorted([len(w) for w in elem]))

[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
